In [2]:
import numpy as np

class Maze(object):
    def __init__(self, filename):
        '''
        Maze objects have two main attributes:
        - dim: mazes should be square, with sides of even length. (integer)
        - walls: passages are coded as a 4-bit number, with a bit value taking
            0 if there is a wall and 1 if there is no wall. The 1s register
            corresponds with a square's top edge, 2s register the right edge,
            4s register the bottom edge, and 8s register the left edge. (numpy
            array)

        The initialization function also performs some consistency checks for
        wall positioning.
        '''
        with open(filename, 'rb') as f_in:

            # First line should be an integer with the maze dimensions
            self.dim = int(f_in.next())

            # Subsequent lines describe the permissability of walls
            walls = []
            for line in f_in:
                walls.append(map(int,line.split(',')))
            self.walls = np.array(walls)

        # Perform validation on maze
        # Maze dimensions
        if self.dim % 2:
            raise Exception('Maze dimensions must be even in length!')
        if self.walls.shape != (self.dim, self.dim):
            raise Exception('Maze shape does not match dimension attribute!')

        # Wall permeability
        wall_errors = []
        # vertical walls
        for x in range(self.dim-1):
            for y in range(self.dim):
                if (self.walls[x,y] & 2 != 0) != (self.walls[x+1,y] & 8 != 0):
                    wall_errors.append([(x,y), 'v'])
        # horizontal walls
        for y in range(self.dim-1):
            for x in range(self.dim):
                if (self.walls[x,y] & 1 != 0) != (self.walls[x,y+1] & 4 != 0):
                    wall_errors.append([(x,y), 'h'])

        if wall_errors:
            for cell, wall_type in wall_errors:
                if wall_type == 'v':
                    cell2 = (cell[0]+1, cell[1])
                    print 'Inconsistent vertical wall betweeen {} and {}'.format(cell, cell2)
                else:
                    cell2 = (cell[0], cell[1]+1)
                    print 'Inconsistent horizontal wall betweeen {} and {}'.format(cell, cell2)
            raise Exception('Consistency errors found in wall specifications!')


    def is_permissible(self, cell, direction):
        """
        Returns a boolean designating whether or not a cell is passable in the
        given direction. Cell is input as a list. Directions may be
        input as single letter 'u', 'r', 'd', 'l', or complete words 'up', 
        'right', 'down', 'left'.
        """
        dir_int = {'u': 1, 'r': 2, 'd': 4, 'l': 8,
                   'up': 1, 'right': 2, 'down': 4, 'left': 8}
        try:
            return (self.walls[tuple(cell)] & dir_int[direction] != 0)
        except:
            print 'Invalid direction provided!'


    def dist_to_wall(self, cell, direction):
        """
        Returns a number designating the number of open cells to the nearest
        wall in the indicated direction. Cell is input as a list. Directions
        may be input as a single letter 'u', 'r', 'd', 'l', or complete words
        'up', 'right', 'down', 'left'.
        """
        dir_move = {'u': [0, 1], 'r': [1, 0], 'd': [0, -1], 'l': [-1, 0],
                    'up': [0, 1], 'right': [1, 0], 'down': [0, -1], 'left': [-1, 0]}

        sensing = True
        distance = 0
        curr_cell = list(cell) # make copy to preserve original
        while sensing:
            if self.is_permissible(curr_cell, direction):
                distance += 1
                curr_cell[0] += dir_move[direction][0]
                curr_cell[1] += dir_move[direction][1]
            else:
                sensing = False
        return distance

In [3]:
testmaze = Maze( 'test_maze_01.txt' )

In [4]:
goal=[(5,5),(5,6),(6,5),(6,6)]
#goal=[(100,100)]

In [14]:
closed=[[0 for col in range(len(testmaze.walls[0]))] for row in range(len(testmaze.walls))]

In [15]:
closed

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [16]:
heuristic=[[-1 for col in range(len(testmaze.walls[0]))] for row in range(len(testmaze.walls))]

In [17]:
for i in range(4):
    heuristic[goal[i][0]][goal[i][1]]=0
    closed[goal[i][0]][goal[i][1]]=1

In [18]:
testmaze.dim

12

In [21]:
for counter in range(50):
    flag=False
    for i in range(testmaze.dim):
        for j in range(testmaze.dim):
            if heuristic[i][j]==-1:
                flag=True
            if heuristic[i][j]==counter:
                if (testmaze.is_permissible((i,j),'up')) and closed[i][j+1]==0:
                    heuristic[i][j+1]=counter+1
                    closed[i][j+1]=1
                if testmaze.is_permissible((i,j),'down') and closed[i][j-1]==0:
                    heuristic[i][j-1]=counter+1
                    closed[i][j-1]=1
                if testmaze.is_permissible((i,j),'left') and closed[i-1][j]==0:
                    heuristic[i-1][j]=counter+1
                    closed[i-1][j]=1
                if testmaze.is_permissible((i,j),'right') and closed[i+1][j]==0:
                    heuristic[i+1][j]=counter+1
                    closed[i+1][j]=1

In [22]:
heuristic

[[30, 29, 28, 27, 26, 25, 24, 25, 24, 25, 24, 23],
 [25, 26, 27, 24, 23, 24, 23, 24, 23, 22, 21, 22],
 [24, 25, 28, 25, 22, 21, 22, 23, 18, 19, 20, 21],
 [23, 26, 27, 26, 19, 20, 21, 22, 17, 16, 15, 14],
 [22, 21, 20, 19, 18, 17, 16, 15, 14, 15, 14, 13],
 [19, 20, 19, 20, 19, 0, 0, 14, 13, 12, 13, 12],
 [18, 17, 18, 19, 20, 0, 0, 13, 12, 11, 12, 11],
 [15, 16, 17, 18, 5, 2, 1, 2, 11, 10, 9, 10],
 [14, 15, 6, 5, 4, 3, 4, 3, 4, 9, 8, 11],
 [13, 14, 15, 6, 5, 6, 5, 6, 5, 6, 7, 12],
 [12, 13, 14, 7, 10, 7, 8, 9, 6, 7, 8, 9],
 [11, 10, 9, 8, 9, 10, 11, 10, 7, 8, 9, 10]]

## Begin search

In [11]:
def search(grid,init,goal,cost):
    
    ### The following definitions are designed for tuples ###
    def equal(a,b):
        # check whether two tuples are equal: a, b are tuples
        if a[0]==b[0] and a[1]==b[1]:
            return True
        else:
            return False
    def add(a,b):
        # a, b, c are tuples
        c=(a[0]+b[0],a[1]+b[1])
        return c
    def tupleInList(a,L):
        # a is a tuple
        # L is a list of tuples
        r=False
        for i in range(len(L)):
            if equal(a,L[i]):
                r=True
        return r
    def direction(delta):
        # from delta tuple (x,y) to direction string, such as:
        # 'up', 'down', 'right', 'left', etc.
        d='left'
        if delta==(0,-1):
            d='down'
        if delta==(1,0):
            d='right'
        if delta==(0,1):
            d='up'
        return d
    def one_norm(a,b):
        d=abs(a[0]-b[0])+abs(a[1]-b[1])
        return d
    
    #########################################################
    closed=[[0 for col in range(len(grid.walls[0]))] for row in range(len(grid.walls))]
    closed[init[0][0]][init[0][1]]=1
    expand=[[-1 for i in range(len(grid.walls[0]))] for j in range(len(grid.walls))] 
    # modify code **********
    action=[[-1]*len(grid.walls[0]) for i in grid.walls]
    heuristic=[[0 for col in range(len(grid.walls[0]))] for row in range(len(grid.walls))]
    for i in range(grid.dim):
        for j in range(grid.dim):
            heuristic[i][j]=min(one_norm((i,j),(grid.dim/2-1,grid.dim/2-1)),
                               one_norm((i,j),(grid.dim/2-1,grid.dim/2)),
                               one_norm((i,j),(grid.dim/2,grid.dim/2-1)),
                               one_norm((i,j),(grid.dim/2,grid.dim/2)))
    
    #no heuristic
    # heuristic=[[0 for col in range(len(grid.walls[0]))] for row in range(len(grid.walls))]
    
    print(heuristic)
    

    
    x=init[0][0]
    y=init[0][1]
    g=0 #g-value
    h=heuristic[x][y]
    f=g+h
    open=[[f,g,h,x,y]]
    
    count=0 # modify code **********

    #set two flag values
    found=False # True: search is complete
    resign=False # True: we caanot find expand
    
    
    
   
        
    while not found and not resign:
        if len(open) == 0:
            resign=True
        else:
            open.sort()
            open.reverse()
            next=open.pop()
            
            
            x=next[3]
            y=next[4]
            g=next[1]

            expand[x][y]=count # modify code **********
            count=count+1 # modify code **********

        if tupleInList((x,y),goal):
            found==True
            goal=[(x,y)] # I add this line *************

        else:
            for i in range(len(delta)):
                x2=x+delta[i][0]
                y2=y+delta[i][1]
                direct=direction(delta[i])
                if x2 >= 0 and x2 < len(grid.walls) and y2 >= 0 and y2 < len(grid.walls[0]):
                    # The above if: should be in the inside of maze.
                    if closed[x2][y2]==0 and grid.is_permissible((x,y),direct):
                        # The above if: never navigate and can be reached.
                        g2=g+cost
                        h2=heuristic[x2][y2]
                        f2=g2+h2
                        open.append([f2,g2,h2,x2,y2])
                        closed[x2][y2]=1
                        action[x2][y2]=i
    print(action)
    policy=[[' ']*len(grid.walls[0]) for i in grid.walls]
    x=goal[0][0]
    y=goal[0][1]
    policy[x][y]='*'
    while x != init[0][0] or y != init[0][1]:
        x2=x-delta[action[x][y]][0]
        y2=y-delta[action[x][y]][1]
        policy[x2][y2]=delta_name[action[x][y]]
        x=x2
        y=y2
    

    return policy


In [12]:
policy=search(testmaze,init,goal,cost)

[[10, 9, 8, 7, 6, 5, 5, 6, 7, 8, 9, 10], [9, 8, 7, 6, 5, 4, 4, 5, 6, 7, 8, 9], [8, 7, 6, 5, 4, 3, 3, 4, 5, 6, 7, 8], [7, 6, 5, 4, 3, 2, 2, 3, 4, 5, 6, 7], [6, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 6], [5, 4, 3, 2, 1, 0, 0, 1, 2, 3, 4, 5], [5, 4, 3, 2, 1, 0, 0, 1, 2, 3, 4, 5], [6, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 6], [7, 6, 5, 4, 3, 2, 2, 3, 4, 5, 6, 7], [8, 7, 6, 5, 4, 3, 3, 4, 5, 6, 7, 8], [9, 8, 7, 6, 5, 4, 4, 5, 6, 7, 8, 9], [10, 9, 8, 7, 6, 5, 5, 6, 7, 8, 9, 10]]
[[-1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 0, 3, 2, 2, 0, 1, 0, 1, 1, 1], [0, 3, 0, 3, 2, 2, 0, 1, 2, 2, 0, 1], [0, 2, 0, 1, 2, 0, 1, 0, 0, 1, 1, 1], [0, 1, 1, 2, 0, 1, 1, 1, 1, 1, 0, 0], [2, 2, 0, 1, 0, -1, -1, 0, 0, 1, 0, 0], [2, 2, 0, 0, 0, -1, 3, 0, 1, 0, 1, 1], [2, 0, 2, 0, 3, 3, 1, 3, 0, 0, 2, 0], [0, 1, 2, 3, 1, 1, 1, 2, 3, 0, 1, 0], [0, 1, 1, 3, 0, 2, 2, 2, 2, 2, 0, 0], [0, 1, 0, 3, 3, 0, 2, 3, 0, 0, 0, 1], [0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 2, 0]]


In [13]:
policy

[['^', '^', '>', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 ['>', 'V', 'V', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 ['>', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 ['>', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 ['^', '^', '>', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', '>', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', '>', 'V', ' ', ' ', ' ', '*', ' ', ' ', ' ', ' ', ' '],
 ['>', 'V', ' ', ' ', ' ', '^', '<', ' ', ' ', ' ', ' ', ' '],
 ['>', ' ', ' ', '^', '^', '<', ' ', ' ', ' ', ' ', ' ', ' '],
 ['>', ' ', ' ', '<', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 ['>', ' ', ' ', '<', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 ['^', '^', '^', '<', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']]

In [14]:
for row in policy:
    print row

['^', '^', '>', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
['>', 'V', 'V', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
['>', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
['>', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
['^', '^', '>', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', '>', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', '>', 'V', ' ', ' ', ' ', '*', ' ', ' ', ' ', ' ', ' ']
['>', 'V', ' ', ' ', ' ', '^', '<', ' ', ' ', ' ', ' ', ' ']
['>', ' ', ' ', '^', '^', '<', ' ', ' ', ' ', ' ', ' ', ' ']
['>', ' ', ' ', '<', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
['>', ' ', ' ', '<', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
['^', '^', '^', '<', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
